
# Importacion

In [ ]:
!pip install tensorflow_addons

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import cv2
import time
import datetime
import random
import pandas as pd
import shutil
from PIL import Image
from entrenamiento import genera_dataframe, almacena_bn,\
    genera_dataframe_bn, borrado_entrenamiento_test,\
    plot_prediction, convertir_jpeg, preprocesado
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score,\
    precision_score, roc_auc_score 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Activation, Dropout,\
    MaxPooling2D
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
import seaborn as sns
import tensorflow_addons as tfa
from keras import backend as K

# Variables

In [ ]:
# Ruta
path_incendios = '/content/gdrive/My Drive/incendios_satelite/Incendios masivos/'
path_no_incendios = '/content/gdrive/My Drive/incendios_satelite/Control/'
path_train_incendio = '/content/gdrive/My Drive/incendios_satelite/Entrenamiento_incendio/'
path_train_no_incendio = '/content/gdrive/My Drive/incendios_satelite/Entrenamiento_no_incendio/'
path_test_incendio = '/content/gdrive/My Drive/incendios_satelite/Validacion_incendio/'
path_test_no_incendio = '/content/gdrive/My Drive/incendios_satelite/Validacion_no_incendio/'
path_incendios = '/content/gdrive/My Drive/incendios_satelite/Incendios masivos/'
path_no_incendios = '/content/gdrive/My Drive/incendios_satelite/Control/'
var_real ='/content/gdrive/My Drive/incendios_satelite/1reales/'
var_lst ='/content/gdrive/My Drive/incendios_satelite/2lst/'
var_nvdi ='/content/gdrive/My Drive/incendios_satelite/3nvdi/'
var_poblacion ='/content/gdrive/My Drive/incendios_satelite/4poblacion/'
var_viento_norte ='/content/gdrive/My Drive/incendios_satelite/5viento_norte/'
var_viento_este ='/content/gdrive/My Drive/incendios_satelite/6viento_este/'
ruta_training = 'training_data/'
ruta_test = 'validation_data/'
ruta_class_0 = 'class_0/'
ruta_class_1 = 'class_1/'

# Genero lista de imágenes y de control

In [ ]:
lst_incendios_train = []
lst_incendios_test = []

# Listado de números
for file in os.listdir(path_incendios):
  if "LST_Day" in file:
    id_imagen = int(file.split("L")[0])
    if random.random() < 0.8:
      lst_incendios_train.append(id_imagen)
    else:
      lst_incendios_test.append(id_imagen)

for file in os.listdir(path_no_incendios):
  if "LST_Day" in file:
    id_imagen = int(file.split("L")[0])
    if random.random() < 0.8:
      lst_incendios_train.append(id_imagen)
    else:
      lst_incendios_test.append(id_imagen)

In [ ]:
# Reparto en carpetas
for file in os.listdir(path_incendios):
  if "LST_Day" in file:
    id_imagen = int(file.split("L")[0])
  elif "ND" in file:
    id_imagen = int(file.split("N")[0])

  if id_imagen in lst_incendios_train:
    shutil.copy(path_incendios + file, path_train_incendio)
  elif id_imagen in lst_incendios_test:
    shutil.copy(path_incendios + file, path_test_incendio)

for file in os.listdir(path_no_incendios):
  if "LST_Day" in file:
    id_imagen = int(file.split("L")[0])
  elif "ND" in file:
    id_imagen = int(file.split("N")[0])

  if id_imagen in lst_incendios_train:
    shutil.copy(path_no_incendios + file, path_train_no_incendio)
  elif id_imagen in lst_incendios_test:
    shutil.copy(path_no_incendios + file, path_test_no_incendio)

# Preprocesado

In [ ]:
# Convertir a JPEG
convertir_jpeg(var_lst + ruta_training + ruta_class_1)
convertir_jpeg(var_lst + ruta_training + ruta_class_0)
convertir_jpeg(var_lst + ruta_test + ruta_class_1)
convertir_jpeg(var_lst + ruta_test + ruta_class_0)
convertir_jpeg(var_nvdi + ruta_training + ruta_class_1)
convertir_jpeg(var_nvdi + ruta_training + ruta_class_0)
convertir_jpeg(var_nvdi + ruta_test + ruta_class_1)
convertir_jpeg(var_nvdi + ruta_test + ruta_class_0)
convertir_jpeg(var_poblacion + ruta_training + ruta_class_1)
convertir_jpeg(var_poblacion + ruta_training + ruta_class_0)
convertir_jpeg(var_poblacion + ruta_test + ruta_class_1)
convertir_jpeg(var_poblacion + ruta_test + ruta_class_0)

# Modelos

## Red Convolucional

# Temperatura

## Preparación de datos

In [ ]:
# DATOS DL
# Generamos el dataframe de temperatura
temp_train_0 = '/content/gdrive/My Drive/incendios_satelite/2lst/training_data/class_0/'
temp_train_1 = '/content/gdrive/My Drive/incendios_satelite/2lst/training_data/class_1/'
temp_test_0 = '/content/gdrive/My Drive/incendios_satelite/2lst/validation_data/class_0/'
temp_test_1 = '/content/gdrive/My Drive/incendios_satelite/2lst/validation_data/class_1/'

inicio = time.time()
df_temp_train, lst_features_x, df_id = genera_dataframe(ruta_incendio = temp_train_1,
                                      ruta_no_incendio = temp_train_0,
                                      radar= "LST_Day",
                                      reshape_x = 23,
                                      reshape_y = 23)
df_temp_test, lst_features_x, df_id = genera_dataframe(ruta_incendio = temp_test_1,
                                      ruta_no_incendio = temp_test_0,
                                      radar= "LST_Day",
                                      reshape_x = 23,
                                      reshape_y = 23)

final = time.time()
print("Ha tardado en generar el dataframe {} minutos".format((final-inicio)/60))

In [ ]:
# DATOS DL
kernel_size = [2, 3, 5, 8]
lst_lr = [0.001, 0.01, 0.1]
lst_epoch = [20, 50, 100, 200]
lado = 23 # Ancho y largo del lado de la imagen
lst_neuronas = [32, 64, 128, 256]

from tensorflow import keras
import tensorflow

train_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/2lst/training_data/',
    labels='inferred', 
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    image_size=(lado, lado))
validation_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/2lst/validation_data/',
    labels='inferred',
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    image_size=(lado, lado))

validation_conv = validation_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))
train_conv = train_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))

train_ds = train_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))
validation_ds = validation_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))

## Modelos ML

In [ ]:
# Generamos procesos previos:
X_train, X_test, y_train, y_test = preprocesado(df_train = df_temp_train, df_test = df_temp_test)

# REGRESIÓN LOGÍSTICA
# Hiperparámetros
C = [0.9, 0.8, 0.7] # Intensidad de la regularización
regularization_methods = ["l1", "l2"] # Métodos de regularización
max_iterations = [100, 1000, 10000, 100000] # Números máximos de iteraciones

temperatura_result_logistica = regresion_logistica(X_train = X_train,
                                              X_test = X_test,
                                              y_train = y_train,
                                              y_test = y_test,
                                              C = C, 
                                regularization_methods = regularization_methods,
                                max_iterations = max_iterations)

# XGBOOST
# Hiperparámetros
learning_rate = [0.001, 0.01, 0.1] # Tasa de aprendizaje
max_depth = [5, 10, 20] # Profundidaz máxima
n_estimators = [10, 50, 100] # Número de árboles
temperatura_result_xgboost = func_xgboost(X_train = X_train,
                                      X_test = X_test,
                                      y_train = y_train,
                                      y_test = y_test,
                                      learning_rate = learning_rate,
                                      max_depth = max_depth,
                                      n_estimators = n_estimators)

In [ ]:
temp_best_result_log = max(
    temperatura_result_logistica, key=lambda x: x['f1_score'])

In [ ]:
temp_best_result_log

In [ ]:
temp_best_result_xgboost = max(
    temperatura_result_xgboost, key=lambda x: x['f1_score'])

In [ ]:
temp_best_result_xgboost

## Modelos DL

In [ ]:
resultados_red_densa_temp = red_densa(train = train_ds,
                                    test = validation_ds,
                                    lst_neuronas = lst_neuronas, 
                                    lst_lr = lst_lr,
                                    lst_epoch = lst_epoch,
                                    input_shape= lado*lado)

In [ ]:
resultados_conv_temp = red_convolucional(train = train_conv,
                                    test = validation_conv,
                                    kernel_size = kernel_size,
                                    lst_lr = lst_lr,
                                    lst_epoch = lst_epoch,
                                    lado = lado)

In [ ]:
temp_best_result_rd = max(
    resultados_red_densa_temp, key=lambda x: x['f1_score'])

In [ ]:
temp_best_result_rd

In [ ]:
temp_best_result_conv = max(
    resultados_conv_temp, key=lambda x: x['f1_score'])

In [ ]:
temp_best_result_conv

In [ ]:
temp_best_result_rd

In [ ]:
temp_best_result_conv

# Humedad relativa

## Preparación de datos

In [ ]:
# DATOS ML
# Generamos el dataframe de humedad relativa
hum_train_0 = '/content/gdrive/My Drive/incendios_satelite/3nvdi/training_data/class_0/'
hum_train_1 = '/content/gdrive/My Drive/incendios_satelite/3nvdi/training_data/class_1/'
hum_test_0 = '/content/gdrive/My Drive/incendios_satelite/3nvdi/validation_data/class_0/'
hum_test_1 = '/content/gdrive/My Drive/incendios_satelite/3nvdi/validation_data/class_1/'

inicio = time.time()
df_hum_train, lst_features_x, df_id = genera_dataframe(ruta_incendio = hum_train_1,
                                      ruta_no_incendio = hum_train_0,
                                      radar= "NDVI",
                                      reshape_x = 23,
                                      reshape_y = 23)
df_hum_test, lst_features_x, df_id = genera_dataframe(ruta_incendio = hum_test_1,
                                      ruta_no_incendio = hum_test_0,
                                      radar= "NDVI",
                                      reshape_x = 23,
                                      reshape_y = 23)

final = time.time()
print("Ha tardado en generar el dataframe {} minutos".format((final-inicio)/60))

In [ ]:
from tensorflow import keras
import tensorflow
# DATOS DL
kernel_size = [2, 3, 5, 8]
lst_lr = [0.001, 0.01, 0.1]
lst_epoch = [20, 50, 100, 200]
lado = 23 # Ancho y largo del lado de la imagen
lst_neuronas = [32, 64, 128, 256]

train_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/3nvdi/training_data/',
    labels='inferred', 
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    image_size=(lado, lado))
validation_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/3nvdi/validation_data/',
    labels='inferred',
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    image_size=(lado, lado))

validation_conv = validation_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))
train_conv = train_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))

train_ds = train_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))
validation_ds = validation_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))

## Modelos ML

In [ ]:
inicio = time.time()
# Generamos procesos previos:
X_train, X_test, y_train, y_test = preprocesado(df_train = df_hum_train, df_test = df_hum_test)

# REGRESIÓN LOGÍSTICA
# Hiperparámetros
C = [0.9, 0.8, 0.7] # Lista de tasas de aprendizaje
regularization_methods = ["l1", "l2"] # Métodos de regularización
max_iterations = [100, 1000, 10000, 100000] # Números máximos de iteraciones

humedad_result_logistica = regresion_logistica(X_train = X_train,
                                              X_test = X_test,
                                              y_train = y_train,
                                              y_test = y_test,
                                              C = C, 
                                regularization_methods = regularization_methods,
                                max_iterations = max_iterations)

# XGBOOST
# Hiperparámetros
learning_rate = [0.001, 0.01, 0.1] # Tasa de aprendizaje
max_depth = [5, 10, 20] # Profundidaz máxima
n_estimators = [10, 50, 100, 200, 300] # Número de árboles
humedad_result_xgboost = func_xgboost(X_train = X_train,
                                      X_test = X_test,
                                      y_train = y_train,
                                      y_test = y_test,
                                      learning_rate = learning_rate,
                                      max_depth = max_depth,
                                      n_estimators = n_estimators)

final = time.time()
print("Ha tardado en generar el dataframe {} minutos".format((final-inicio)/60))

In [ ]:
humedad_best_result_log = max(
    humedad_result_logistica, key=lambda x: x['f1_score'])
humedad_best_result_xgboost = max(
    humedad_result_xgboost, key=lambda x: x['f1_score'])

In [ ]:
humedad_best_result_log

In [ ]:
humedad_best_result_xgboost

## Modelos DL

In [ ]:
lst_neuronas = [128]
resultados_red_densa = red_densa(train = train_ds,
                                    test = validation_ds,
                                    lst_neuronas = lst_neuronas, 
                                    lst_lr = lst_lr,
                                    lst_epoch = lst_epoch,
                                    input_shape= lado*lado)

In [ ]:
resultados_conv = red_convolucional(train = train_conv,
                                    test = validation_conv,
                                    kernel_size = kernel_size,
                                    lst_lr = lst_lr,
                                    lst_epoch = lst_epoch,
                                    lado = lado)

In [ ]:
hum_best_result_rd = max(
    resultados_red_densa, key=lambda x: x['f1_score'])

In [ ]:
hum_best_result_conv = max(
    resultados_conv, key=lambda x: x['f1_score'])

In [ ]:
hum_best_result_rd

In [ ]:
hum_best_result_conv

# Población

## Preparación de datos

In [ ]:
# Generamos el dataframe de temperatura
inicio = time.time()
pob_train_0 = '/content/gdrive/My Drive/incendios_satelite/4poblacion/training_data/class_0/'
pob_train_1 = '/content/gdrive/My Drive/incendios_satelite/4poblacion/training_data/class_1/'
pob_test_0 = '/content/gdrive/My Drive/incendios_satelite/4poblacion/validation_data/class_0/'
pob_test_1 = '/content/gdrive/My Drive/incendios_satelite/4poblacion/validation_data/class_1/'

df_pob_train, lst_features_x, df_id = genera_dataframe(ruta_incendio = pob_train_1,
                                      ruta_no_incendio = pob_train_0,
                                      radar= "smod",
                                      reshape_x = 23,
                                      reshape_y = 23)
df_pob_test, lst_features_x, df_id = genera_dataframe(ruta_incendio = pob_test_1,
                                      ruta_no_incendio = pob_test_0,
                                      radar= "smod",
                                      reshape_x = 23,
                                      reshape_y = 23)

final = time.time()
print("Ha tardado en generar el dataframe {} minutos".format((final-inicio)/60))

In [ ]:
# DATOS DL
import tensorflow
from tensorflow import keras

kernel_size = [2, 3, 5, 8]
lst_lr = [0.001, 0.01, 0.1]
lst_epoch = [20, 50, 100, 200]
lado = 23 # Ancho y largo del lado de la imagen
lst_neuronas = [32, 64, 128, 256]

train_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/4poblacion/training_data/',
    labels='inferred', 
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    seed=123,
    image_size=(lado, lado))
validation_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/4poblacion/validation_data/',
    labels='inferred',
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    seed=123,
    image_size=(lado, lado))

validation_conv = validation_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))
train_conv = train_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))

train_ds = train_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))
validation_ds = validation_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))

## Modelos ML

In [ ]:
inicio = time.time()
# Generamos procesos previos:
X_train, X_test, y_train, y_test = preprocesado(df_train = df_pob_train, df_test = df_pob_test)

# REGRESIÓN LOGÍSTICA
# Hiperparámetros
C = [0.9, 0.8, 0.7] # Lista de tasas de aprendizaje
regularization_methods = ["l1", "l2"] # Métodos de regularización
max_iterations = [100, 1000, 10000, 10000] # Números máximos de iteraciones

poblacion_result_logistica = regresion_logistica(X_train = X_train,
                                              X_test = X_test,
                                              y_train = y_train,
                                              y_test = y_test,
                                              C = C, 
                                regularization_methods = regularization_methods,
                                max_iterations = max_iterations)

# XGBOOST
# Hiperparámetros
learning_rate = [0.001, 0.01, 0.1] # Tasa de aprendizaje
max_depth = [5, 10, 20] # Profundidaz máxima
n_estimators = [10, 50, 100, 200, 300, 400] # Número de árboles
poblacion_result_xgboost = func_xgboost(X_train = X_train,
                                      X_test = X_test,
                                      y_train = y_train,
                                      y_test = y_test,
                                      learning_rate = learning_rate,
                                      max_depth = max_depth,
                                      n_estimators = n_estimators)

final = time.time()
print("Ha tardado en generar el dataframe {} minutos".format((final-inicio)/60))

In [ ]:
poblacion_best_result_log = max(
    poblacion_result_logistica, key=lambda x: x['f1_score'])
poblacion_best_result_xgboost = max(
    poblacion_result_xgboost, key=lambda x: x['f1_score'])

In [ ]:
poblacion_best_result_log

In [ ]:
poblacion_best_result_xgboost

## Modelos DL

In [ ]:
resultados_red_densa = red_densa(train = train_ds,
                                    test = validation_ds,
                                    lst_neuronas = lst_neuronas, 
                                    lst_lr = lst_lr,
                                    lst_epoch = lst_epoch,
                                    input_shape= lado*lado)

In [ ]:
resultados_conv = red_convolucional(train = train_conv,
                                    test = validation_conv,
                                    kernel_size = kernel_size,
                                    lst_lr = lst_lr,
                                    lst_epoch = lst_epoch,
                                    lado = lado)

In [ ]:
pob_best_result_rd = max(
    resultados_red_densa, key=lambda x: x['f1_score'])

In [ ]:
pob_best_result_conv = max(
    resultados_conv, key=lambda x: x['f1_score'])

In [ ]:
pob_best_result_rd

In [ ]:
pob_best_result_conv

In [ ]:
resultados_red_densa

# Velocidad del viento

## Análisis de distribución

In [ ]:

import math

def promedio(lst):
    return sum(lst) / len(lst)

inicio = time.time()
lst_archivos_incendios = []
lst_archivos_no_incendios = []
radar = "wind"
reshape_x = 23
reshape_y = 23

# Listado de incendios
lista_ficheros_incendios = []
n_varibles_x =reshape_x * reshape_y
print("Creando lista de incendios a leer...")
for file in os.listdir(path_incendios):
      if radar in file:
            lst_archivos_incendios.append(file)

for file in os.listdir(path_no_incendios):
      if radar in file:
            lst_archivos_no_incendios.append(file)  

lista_viento = []

n_varibles_x = reshape_x * reshape_y

for i in lst_archivos_incendios:
    if "north" in i:
        token = i.split("wind")[0]
    
        for j in lst_archivos_incendios:
            if j.split("wind")[0] == token and "east" in j:
                lst_item = []
                imagen1 = cv2.imread(path_incendios + i
                                )[0:reshape_x, 0:reshape_y, :]
                image_grey1 = np.array(cv2.cvtColor(imagen1, cv2.COLOR_BGR2GRAY))
                image_grey_reshape_1 = image_grey1.reshape(n_varibles_x)
                lst_grey_reshape_1 = image_grey_reshape_1.tolist()
                viento_u = promedio(lst_grey_reshape_1)
    
                imagen2 = cv2.imread(path_incendios + j
                                )[0:reshape_x, 0:reshape_y, :]
                image_grey2 = np.array(cv2.cvtColor(imagen2, cv2.COLOR_BGR2GRAY))
                image_grey_reshape_2 = image_grey2.reshape(n_varibles_x)
                lst_grey_reshape_2 = image_grey_reshape_2.tolist()
                viento_v = promedio(lst_grey_reshape_2)
            
                viento = math.sqrt(viento_u**2 + viento_v**2)
                id = int(i.split('w')[0])
                lst_item.append(viento)
                lst_item.append(1)
                lst_item.append(id)
                lista_viento.append(lst_item)
            
for i in lst_archivos_no_incendios:
    if "north" in i:
        token = i.split("wind")[0]
    
        for j in lst_archivos_no_incendios:
            if j.split("wind")[0] == token and "east" in j:
                lst_item = []
                imagen1 = cv2.imread(path_no_incendios + i
                                )[0:reshape_x, 0:reshape_y, :]
                image_grey1 = np.array(cv2.cvtColor(imagen1, cv2.COLOR_BGR2GRAY))
                image_grey_reshape_1 = image_grey1.reshape(n_varibles_x)
                lst_grey_reshape_1 = image_grey_reshape_1.tolist()
                viento_u = promedio(lst_grey_reshape_1)
            
                imagen2 = cv2.imread(path_no_incendios + j
                                )[0:reshape_x, 0:reshape_y, :]
                image_grey2 = np.array(cv2.cvtColor(imagen2, cv2.COLOR_BGR2GRAY))
                image_grey_reshape_2 = image_grey2.reshape(n_varibles_x)
                lst_grey_reshape_2 = image_grey_reshape_2.tolist()
                viento_v = promedio(lst_grey_reshape_2)
            
                viento = math.sqrt(viento_u**2 + viento_v**2)
                id = int(i.split('w')[0])
                         
                lst_item.append(viento)
                lst_item.append(0)
                lst_item.append(id)
                lista_viento.append(lst_item)


columns = ["col_x","target","id"]
  # Genero el dataframe en sí
df_viento = pd.DataFrame(data = lista_viento, columns =columns )

# Barajo el dataframe
df_viento=df_viento.sample(frac=1).reset_index(drop=True)

final = time.time()
print("Ha tardado en generar el dataframe {} minutos".format((final-inicio)/60))
df_viento.head()

In [ ]:
# Genero gráfico de cajas y bigotes
import matplotlib.pyplot as plt
import numpy as np

# Supongamos que tenemos los siguientes datos:
velocidad_viento_0 = df_viento[df_viento["target"] == 0]["col_x"].to_list()
velocidad_viento_1 = df_viento[df_viento["target"] == 1]["col_x"].to_list()

# Creamos el gráfico de barras y bigotes
fig, ax = plt.subplots()

# Creamos los bigotes
bigotes = np.array([[np.min(velocidad_viento_0), np.max(velocidad_viento_0)], 
                    [np.min(velocidad_viento_1), np.max(velocidad_viento_1)]])

# Creamos las barras
barras = np.array([np.mean(velocidad_viento_0), np.mean(velocidad_viento_1)])

# Dibujamos el gráfico
plt.boxplot([velocidad_viento_0, velocidad_viento_1])

# Añadimos etiquetas ejes y título
plt.xlabel('Target')
plt.ylabel('Velocidad del viento')
plt.title('Boxplot velocidad del viento')
plt.xticks([1, 2], [0, 1])

# Mostramos el gráfico
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Creamos el primer histograma
plt.hist(velocidad_viento_0, label='target = 0')

# Creamos el segundo histograma
plt.hist(velocidad_viento_1, label='target = 1')

# Añadimos etiquetas ejes y título
plt.xlabel('Velocidad del viento')
plt.ylabel('Frecuencia')
plt.title('Distribución velocidad del viento')

# Añadimos la leyenda
plt.legend()

# Mostramos el gráfico
plt.show()

In [ ]:
# Grabo el resultado
df_viento.to_csv('/content/gdrive/My Drive/incendios_satelite/df_viento.csv', index=False)

# Imagen real

## Preparación de datos

In [ ]:
# Generamos el dataframe de las imágenes reales
real_train_0 = '/content/gdrive/My Drive/incendios_satelite/1reales/training_data/class_0/'
real_train_1 = '/content/gdrive/My Drive/incendios_satelite/1reales/training_data/class_1/'
real_test_0 = '/content/gdrive/My Drive/incendios_satelite/1reales/validation_data/class_0/'
real_test_1 = '/content/gdrive/My Drive/incendios_satelite/1reales/validation_data/class_1/'

inicio = time.time()
df_real_train, lst_features_x, df_id = genera_dataframe(ruta_incendio = real_train_1,
                                      ruta_no_incendio = real_train_0,
                                      radar= "incendio",
                                      reshape_x = 2100,
                                      reshape_y = 2100)
df_real_test, lst_features_x, df_id = genera_dataframe(ruta_incendio = real_test_1,
                                      ruta_no_incendio = real_test_0,
                                      radar= "incendio",
                                      reshape_x = 2100,
                                      reshape_y = 2100)

final = time.time()
print("Ha tardado en generar el dataframe {} minutos".format((final-inicio)/60))

In [ ]:
# DATOS DL
import tensorflow
from tensorflow import keras

kernel_size = [2, 3, 5, 8]
lst_lr = [0.001]
lst_epoch = [20, 50, 100, 200]
lado = 23 # Ancho y largo del lado de la imagen
lst_neuronas = [32, 64, 128, 256]
# lst_lr = [0.001, 0.01, 0.1]
train_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/1reales/training_data/',
    labels='inferred', 
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    seed=123,
    image_size=(lado, lado))
validation_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/1reales/validation_data/',
    labels='inferred',
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    seed=123,
    image_size=(lado, lado))

validation_conv = validation_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))
train_conv = train_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))

train_ds = train_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))
validation_ds = validation_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))

## Modelos ML

In [ ]:
# Generamos procesos previos:
inicio = time.time()
X_train, X_test, y_train, y_test = preprocesado(df_train = df_real_train, df_test = df_real_test)

# REGRESIÓN LOGÍSTICA
# Hiperparámetros
C = [0.9, 0.8, 0.7] # Intensidad de la regularización
regularization_methods = ["l1", "l2"] # Métodos de regularización
max_iterations = [100, 1000, 10000, 10000] # Números máximos de iteraciones

real_result_logistica = regresion_logistica(X_train = X_train,
                                              X_test = X_test,
                                              y_train = y_train,
                                              y_test = y_test,
                                              C = C, 
                                regularization_methods = regularization_methods,
                                max_iterations = max_iterations)

# XGBOOST
# Hiperparámetros
learning_rate = [0.001, 0.01, 0.1] # Tasa de aprendizaje
max_depth = [5, 10, 20] # Profundidaz máxima
n_estimators = [10, 50, 100, 200] # Número de árboles
real_result_xgboost = func_xgboost(X_train = X_train,
                                      X_test = X_test,
                                      y_train = y_train,
                                      y_test = y_test,
                                      learning_rate = learning_rate,
                                      max_depth = max_depth,
                                      n_estimators = n_estimators)

final = time.time()
print("Ha tardado en generar el dataframe {} minutos".format((final-inicio)/60))

In [ ]:
real_best_result_log = max(
    real_result_logistica, key=lambda x: x['f1_score'])
real_best_result_xgboost = max(
    real_result_xgboost, key=lambda x: x['f1_score'])

In [ ]:
real_best_result_log

In [ ]:
real_best_result_xgboost

## Modelos de DL

In [ ]:
# Falta 0.01

lst_lr = [0.001]
lst_epoch = [20, 50, 100, 200]
lado = 23 # Ancho y largo del lado de la imagen
lst_neuronas = [256]
resultados_red_densa = red_densa(train = train_ds,
                                    test = validation_ds,
                                    lst_neuronas = lst_neuronas, 
                                    lst_lr = lst_lr,
                                    lst_epoch = lst_epoch,
                                    input_shape= lado*lado)

In [ ]:
kernel_size = [2, 3, 5, 8]
lst_epoch = [20, 50, 100, 200]
lst_lr = [0.001, 0.01, 0.1]
resultados_conv = red_convolucional(train = train_conv,
                                    test = validation_conv,
                                    kernel_size = kernel_size,
                                    lst_lr = lst_lr,
                                    lst_epoch = lst_epoch,
                                    lado = lado)

In [ ]:
real_best_result_rd = max(
    resultados_red_densa, key=lambda x: x['f1_score'])

In [ ]:
resultados_conv_real = max(
    resultados_conv, key=lambda x: x['f1_score'])

In [ ]:
real_best_result_rd

In [ ]:
real_best_result_conv

# Coordenadas X e Y

## Preparación de datos

In [ ]:
# Se ha ejecutado tras el método unificado por lo que ya están creadas las variables
df1 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/total_incendios_control.csv', sep=',')
df2 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/incendios_target3.csv', sep=',')
df1 = df1[["X","Y","id"]]
df2 = df2[["X","Y","id"]]
df_coord = pd.concat([df1,df2], axis = 0)
df_coord = df_coord.drop_duplicates()

In [ ]:
df_train_0 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_training_dataclass_0.csv', sep=',')
df_train_coord_0 = pd.merge(df_coord, df_train_0, how = 'inner', left_on = 'id', right_on = 'id')
df_train_1 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_training_dataclass_1.csv', sep=',')
df_train_coord_1 = pd.merge(df_coord, df_train_1, how = 'inner', left_on = 'id', right_on = 'id')
df_train = pd.concat([df_train_0, df_train_1], axis=0)
df_train = df_train.drop_duplicates()

df_test_0 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_validation_dataclass_0.csv', sep=',')
df_test_coord_0 = pd.merge(df_coord, df_test_0, how = 'inner', left_on = 'id', right_on = 'id')
df_test_1 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_validation_dataclass_1.csv', sep=',')
df_test_coord_1 = pd.merge(df_coord, df_test_1, how = 'inner', left_on = 'id', right_on = 'id')
df_test = pd.concat([df_test_0, df_test_1], axis=0)
df_test = df_test.drop_duplicates()

df_train_coord_0["target"] = 0
df_train_coord_1["target"] = 1
df_test_coord_0["target"] = 0
df_test_coord_1["target"] = 1
df_coordenadas = pd.concat([df_train_coord_0, df_train_coord_1, df_test_coord_0, df_test_coord_1])
df_coordenadas_train = pd.concat([df_train_coord_0, df_train_coord_1])
df_coordenadas_test= pd.concat([df_test_coord_0, df_test_coord_1])

In [ ]:
df_coordenadas.to_csv('/content/gdrive/My Drive/incendios_satelite/df_coordenadas.csv', index=False)

## Modelos de ML

In [ ]:
# Procesos previos
# Separamos los datos
var_x = ["X","Y"]
X_train = df_coordenadas_train[var_x]
y_train = df_coordenadas_train["target"]
X_test = df_coordenadas_test[var_x]
y_test = df_coordenadas_test["target"]

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Hiperparámetros
C = [0.9, 0.8, 0.7] # Intensidad de la regularización
regularization_methods = ["l1", "l2"] # Métodos de regularización
max_iterations = [100, 1000, 10000, 10000, 10000] # Números máximos de iteraciones
learning_rate = [0.001, 0.01, 0.1] # Tasa de aprendizaje
max_depth = [5, 10, 20] # Profundidaz máxima
n_estimators = [10, 50, 100, 200] # Número de árboles

for lr in  learning_rate:
  for md in max_depth:
    for ne in n_estimators:
      model = xgb.XGBClassifier(objective='binary:logistic', 
                                            learning_rate = lr,
                                            max_depth = md,
                                            n_estimators=ne,
                                            seed=112)

      # Entrenamos el modelo
      model.fit(X_train, y_train)

      # Predecimos los datos de train
      y_pred = model.predict(X_test)

      # Calculamos los estadísticos
      accuracy = accuracy_score(y_test, y_pred)
      precision = precision_score(y_test, y_pred)
      recall = recall_score(y_test, y_pred)
      f1 = f1_score(y_test, y_pred)
      print("Para los valores XGBoost de learning rate {0}, max_depth {1} y n_estimator {2}, el valor de accuracy es: {3}, precision {4}, recall {5} y f1 score {6}".format(
          lr, md, ne, accuracy, precision, recall, f1
      ))

for c in  C:
  for rm in regularization_methods:
    for it in max_iterations:
      log_reg = LogisticRegression(C=c,
                                         penalty=rm,
                                         solver='liblinear',
                                         max_iter=it,
                                         random_state =112)
										 
      # Entrena el modelo con el conjunto de entrenamiento
      log_reg.fit(X_train, y_train)

      # Realiza predicciones con el conjunto de prueba
      y_pred = log_reg.predict(X_test)

      # Calculamos los estadísticos
      accuracy = accuracy_score(y_test, y_pred)
      precision = precision_score(y_test, y_pred)
      recall = recall_score(y_test, y_pred)
      f1 = f1_score(y_test, y_pred)
      print("Para los valores de Regresión Logística de itraciones {0}, con valor de C {1} y regularización {2}, el valor de accuracy es: {3}, precision {4}, recall {5} y f1 score {6}".format(
          it, c, rm, accuracy, precision, recall, f1
      ))

## Modelo de DL

In [ ]:
# Separamos las variables independientes de la variable dependiente
var_x = ["X","Y"]
X_train = df_train[var_x]
y_train = df_train["target"]
X_test = df_test[var_x]
y_test = df_test["target"]

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
lst_lr = [0.01, 0.1]
lst_epoch = [20, 50, 100, 200]
lst_neuronas = [32, 64, 128, 256]
for neuronas in lst_neuronas:
  for lr in lst_lr:
    for epochs in lst_epoch:
      model1 = Sequential()
      model1.add(Dense(neuronas, activation='relu', input_shape=(2,)))
      model1.add(Dropout(0.2))
      model1.add(Dense(neuronas, activation='relu'))
      model1.add(Dropout(0.2))
      model1.add(Dense(1, activation='sigmoid'))

      # Compile
      loss_fn = keras.losses.BinaryCrossentropy()
      opt = keras.optimizers.Adam(learning_rate = lr)
      model1.compile(optimizer=opt, loss=loss_fn,
        metrics=['accuracy',
                tfa.metrics.F1Score(
                num_classes=2,average="micro",
                threshold=0.75)])

      model_dense = model1.fit(X_train, y_train, epochs=epochs, 
                               validation_data=(X_train, y_train),
                               verbose = 0)
                
      val_accuracy = model_dense.history["val_accuracy"][epochs-1]
      val_f1_score = model_dense.history["val_f1_score"][epochs-1]

      print("learning rate {0}, epochs {1} neuronas {2} accuracy {3} y f1_score {4}".format(
        lr, epochs, neuronas, val_accuracy, val_f1_score
                  ))

# Modelo únicos

In [ ]:
kernel_size = [2, 3, 5, 8]
lst_lr = [0.001, 0.01, 0.1]
lst_epoch = [20, 50, 100, 200]
lado = 23 # Ancho y largo del lado de la imagen
lst_neuronas = [32, 64, 128, 256]

In [ ]:
import keras_metrics
import keras

train_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/2lst/training_data/',
    labels='inferred', 
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    image_size=(lado, lado))
validation_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/2lst/validation_data/',
    labels='inferred',
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    image_size=(lado, lado))

validation_conv = train_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))
train_conv = train_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))

train_ds = train_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))
validation_ds = validation_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))

# Temperatura
ks = 3
lr = 0.001
epochs = 200

model2 = Sequential()
model2.add(Conv2D(16, kernel_size=(ks, ks), activation='relu',
                              input_shape=(lado, lado, 1)))
model2.add(Conv2D(32, (ks, ks), activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.2))
model2.add(Conv2D(64, (ks, ks), activation='relu', padding='same'))
model2.add(Conv2D(64, (ks, ks), activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.2))
model2.add(Flatten())
model2.add(Dense(1, activation='sigmoid'))

# Compile
loss_fn = keras.losses.BinaryCrossentropy()
opt = keras.optimizers.Adam(learning_rate = lr)
model2.compile(optimizer=opt, loss=loss_fn,
                            metrics=['accuracy',
                                    tfa.metrics.F1Score(
                                        num_classes=2,average="micro",
                                        threshold=0.75),
                                     keras.metrics.Precision(),
                                     keras.metrics.Recall(),
                                     keras.metrics.AUC()])
tfa.metrics.multilabel_confusion_matrix
                                                     
model_conv = model2.fit(train_conv,
                                    epochs=epochs, 
                                     validation_data=validation_conv)


In [ ]:
import keras_metrics
import keras
lado = 23
# Humedad relativa
train_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/3nvdi/training_data/',
    labels='inferred', 
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    image_size=(lado, lado))
validation_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/3nvdi/validation_data/',
    labels='inferred',
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    image_size=(lado, lado))

validation_conv = train_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))
train_conv = train_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))

train_ds = train_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))
validation_ds = validation_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))

ks = 3
lr = 0.001
epochs = 200

model2 = Sequential()
model2.add(Conv2D(16, kernel_size=(ks, ks), activation='relu',
                              input_shape=(lado, lado, 1)))
model2.add(Conv2D(32, (ks, ks), activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.2))
model2.add(Conv2D(64, (ks, ks), activation='relu', padding='same'))
model2.add(Conv2D(64, (ks, ks), activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.2))
model2.add(Flatten())
model2.add(Dense(1, activation='sigmoid'))

# Compile
loss_fn = keras.losses.BinaryCrossentropy()
opt = keras.optimizers.Adam(learning_rate = lr)
model2.compile(optimizer=opt, loss=loss_fn,
                            metrics=['accuracy',
                                    tfa.metrics.F1Score(
                                        num_classes=2,average="micro",
                                        threshold=0.75),
                                     keras.metrics.Precision(),
                                     keras.metrics.Recall(),
                                     keras.metrics.AUC()])
tfa.metrics.multilabel_confusion_matrix
                                                     
model_conv = model2.fit(train_conv,
                                    epochs=epochs, 
                                     validation_data=validation_conv)

In [ ]:
model2.save('/content/gdrive/My Drive/incendios_satelite/humedad.h5')
plot_prediction(200,model_conv)

In [ ]:
import keras_metrics
import keras
# Población
train_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/4poblacion/training_data/',
    labels='inferred', 
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    seed=123,
    image_size=(lado, lado))
validation_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/4poblacion/validation_data/',
    labels='inferred',
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    seed=123,
    image_size=(lado, lado))

validation_conv = train_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))
train_conv = train_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))

train_ds = train_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))
validation_ds = validation_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))

ks = 5
lr = 0.001
epochs = 200

model2 = Sequential()
model2.add(Conv2D(16, kernel_size=(ks, ks), activation='relu',
                              input_shape=(lado, lado, 1)))
model2.add(Conv2D(32, (ks, ks), activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.2))
model2.add(Conv2D(64, (ks, ks), activation='relu', padding='same'))
model2.add(Conv2D(64, (ks, ks), activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.2))
model2.add(Flatten())
model2.add(Dense(1, activation='sigmoid'))

# Compile
loss_fn = keras.losses.BinaryCrossentropy()
opt = keras.optimizers.Adam(learning_rate = lr)
model2.compile(optimizer=opt, loss=loss_fn,
                            metrics=['accuracy',
                                    tfa.metrics.F1Score(
                                        num_classes=2,average="micro",
                                        threshold=0.75),
                                     keras.metrics.Precision(),
                                     keras.metrics.Recall(),
                                     keras.metrics.AUC()])
tfa.metrics.multilabel_confusion_matrix
                                                     
model_conv = model2.fit(train_conv,
                                    epochs=epochs, 
                                     validation_data=validation_conv)

In [ ]:
model2.save('/content/gdrive/My Drive/incendios_satelite/poblcacion.h5')
plot_prediction(200,model_conv)

In [ ]:
import tensorflow
from tensorflow import keras

lado = 23

# Imagen real
train_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/1reales/training_data/',
    labels='inferred', 
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    seed=123,
    image_size=(lado, lado))
validation_conv = keras.utils.image_dataset_from_directory(
    directory='/content/gdrive/My Drive/incendios_satelite/1reales/validation_data/',
    labels='inferred',
    label_mode='binary',
    color_mode = 'grayscale',
    batch_size=64,
    shuffle=True,
    seed=123,
    image_size=(lado, lado))

validation_conv = train_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))
train_conv = train_conv.map(lambda x, y: (tensorflow.divide(x, 255), y))

train_ds = train_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))
validation_ds = validation_conv.map(lambda x, y: (keras.layers.Flatten()(x), y))


ks = 2
lr = 0.001
epochs = 200

model2 = Sequential()
model2.add(Conv2D(16, kernel_size=(ks, ks), activation='relu',
                              input_shape=(lado, lado, 1)))
model2.add(Conv2D(32, (ks, ks), activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.2))
model2.add(Conv2D(64, (ks, ks), activation='relu', padding='same'))
model2.add(Conv2D(64, (ks, ks), activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.2))
model2.add(Flatten())
model2.add(Dense(1, activation='sigmoid'))

# Compile
loss_fn = keras.losses.BinaryCrossentropy()
opt = keras.optimizers.Adam(learning_rate = lr)
model2.compile(optimizer=opt, loss=loss_fn,
                            metrics=['accuracy',
                                    tfa.metrics.F1Score(
                                        num_classes=2,average="micro",
                                        threshold=0.75),
                                     keras.metrics.Precision(),
                                     keras.metrics.Recall(),
                                     keras.metrics.AUC()])
tfa.metrics.multilabel_confusion_matrix
                                                     
model_conv = model2.fit(train_conv,
                                    epochs=epochs, 
                                     validation_data=validation_conv)

In [ ]:
model2.save('/content/gdrive/My Drive/incendios_satelite/real.h5')
plot_prediction(200,model_conv)

# Union de DF entrenamiento y test

In [ ]:
path_ruta = '/content/gdrive/My Drive/incendios_satelite/'
path_temperatura = '2lst'
path_humedad = '3nvdi'
path_poblacion = '4poblacion'
path_real = '1reales'
path_entrenamiento = 'training_data'
path_test = 'validation_data'
path_clase_0 = 'class_0'
path_clase_1 = 'class_1'

lst_variables = [path_temperatura, path_humedad, path_poblacion, path_real]
lst_tipo = [path_entrenamiento, path_test]
lst_clase = [path_clase_0, path_clase_1]

In [ ]:
inicio = time.time()
for tipo in lst_tipo:
  for clase in lst_clase:
    temperatura =  []
    humedad = []
    poblacion = []
    real = []
    for file in os.listdir(path_ruta + lst_variables[0] + '/' + tipo + '/' + clase + '/'):
        if "jpeg" in file:
          id_imagen = int(file.split("L")[0])
          temperatura.append(id_imagen)
    for file in os.listdir(path_ruta + lst_variables[1] + '/' + tipo + '/' + clase + '/'):
        if "jpeg" in file:
          id_imagen = int(file.split("N")[0])
          humedad.append(id_imagen)
    for file in os.listdir(path_ruta + lst_variables[2] + '/' + tipo + '/' + clase + '/'):
        if "jpeg" in file:
          id_imagen = int(file.split("s")[0])
          poblacion.append(id_imagen)
    for file in os.listdir(path_ruta + lst_variables[3] + '/' + tipo + '/' + clase + '/'):
        if "jpeg" in file:
          id_imagen = int(file.split("i")[0])
          real.append(id_imagen)
        
    # Genero los dataframes
    df_temperatura = pd.DataFrame(data = temperatura, columns =["id"] )
    df_humedad = pd.DataFrame(data = humedad, columns =["id"] )
    df_poblacion = pd.DataFrame(data = poblacion, columns =["id"] )
    df_real = pd.DataFrame(data = real, columns =["id"] )

    # Me quedo con los que estén en todos
    df_cruce = pd.merge(df_temperatura, df_humedad, how = 'inner', left_on = 'id', right_on = 'id')
    df_cruce = pd.merge(df_cruce, df_poblacion, how = 'inner', left_on = 'id', right_on = 'id')
    df_cruce = pd.merge(df_cruce, df_real, how = 'inner', left_on = 'id', right_on = 'id')
    df_cruce = df_cruce.drop_duplicates()

    # Escribo en csv para poder reutilizarlo
    archivo = 'df_' + tipo + clase + '.csv'
    print("Guardo el fichero {0} que contiene {1} registros".format(archivo, len(df_cruce)))
    df_cruce.to_csv('/content/gdrive/My Drive/incendios_satelite/' + archivo, index=False)

final = time.time()
print("Ha tardado en generar el dataframe {} minutos".format((final-inicio)/60))

## Temperatura

In [ ]:
df_train_0 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_training_dataclass_0.csv', sep=',')
df_train_1 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_training_dataclass_1.csv', sep=',')
df_train = pd.concat([df_train_0, df_train_1], axis=0)
df_train = df_train.drop_duplicates()

df_test_0 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_validation_dataclass_0.csv', sep=',')
df_test_1 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_validation_dataclass_1.csv', sep=',')
df_test = pd.concat([df_test_0, df_test_1], axis=0)
df_test = df_test.drop_duplicates()

# DATOS DL
# Generamos el dataframe de temperatura
inicio = time.time()
path_no_incendios = path_ruta + lst_variables[0] + '/' + lst_tipo[0] + '/' + lst_clase[0] + '/'
path_incendios = path_ruta + lst_variables[0] + '/' + lst_tipo[0] + '/' + lst_clase[1] + '/'
path_no_incendios_test = path_ruta + lst_variables[0] + '/' + lst_tipo[1] + '/' + lst_clase[0] + '/'
path_incendios_test = path_ruta + lst_variables[0] + '/' + lst_tipo[1] + '/' + lst_clase[1] + '/'
print(path_incendios_test)
df_temperatura, lst_features_x, df_id = genera_dataframe(ruta_incendio = path_incendios,
                                      ruta_no_incendio = path_no_incendios,
                                      radar= "LST_Day",
                                      reshape_x = 23,
                                      reshape_y = 23)

df_temperatura_test, lst_features_x, df_id_test = genera_dataframe(ruta_incendio = path_incendios_test,
                                      ruta_no_incendio = path_no_incendios_test,
                                      radar= "LST_Day",
                                      reshape_x = 23,
                                      reshape_y = 23)
final = time.time()
print("Ha tardado en generar el dataframe {} minutos".format((final-inicio)/60))

In [ ]:
# Procesos previos
# Separamos los datos
X_train = df_temperatura[lst_features_x]
y_train = df_temperatura["target"]
X_test = df_temperatura_test[lst_features_x]
y_test = df_temperatura_test["target"]

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

model = xgb.XGBClassifier(objective='binary:logistic', 
                                      learning_rate = 0.1,
                                      max_depth = 5,
                                      n_estimators=200,
                                      seed=112)

# Entrenamos el modelo
model.fit(X_train, y_train)

# Predecimos los datos de train
y_pred = model.predict(X_test)

# Calculamos los estadísticos
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("El valor de accuracy es: {0}, precision {1}, recall {2} y f1 score {3}".format(
    accuracy, precision, recall, f1
))
df_temperatura_test['predicciones_temp'] = y_pred
df_temperatura_test['target_temp'] = y_test
df_temperatura_test = df_temperatura_test[["id","predicciones_temp","target_temp"]]
df_temperatura_test.head()

In [ ]:
df_temperatura_test = pd.merge(df_temperatura_test, df_test, how = 'inner', left_on = 'id', right_on = 'id')

In [ ]:
df_temperatura_test.to_csv('/content/gdrive/My Drive/incendios_satelite/df_temperatura_XGBoost.csv', index=False)

##  Humedad relativa

In [ ]:
df_train_0 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_training_dataclass_0.csv', sep=',')
df_train_1 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_training_dataclass_1.csv', sep=',')
df_train = pd.concat([df_train_0, df_train_1], axis=0)
df_train = df_train.drop_duplicates()

df_test_0 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_validation_dataclass_0.csv', sep=',')
df_test_1 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_validation_dataclass_1.csv', sep=',')
df_test = pd.concat([df_test_0, df_test_1], axis=0)
df_test = df_test.drop_duplicates()

# DATOS DL
# Generamos el dataframe de humedad relativa
inicio = time.time()
path_no_incendios = path_ruta + lst_variables[1] + '/' + lst_tipo[0] + '/' + lst_clase[0] + '/'
path_incendios = path_ruta + lst_variables[1] + '/' + lst_tipo[0] + '/' + lst_clase[1] + '/'
path_no_incendios_test = path_ruta + lst_variables[1] + '/' + lst_tipo[1] + '/' + lst_clase[0] + '/'
path_incendios_test = path_ruta + lst_variables[1] + '/' + lst_tipo[1] + '/' + lst_clase[1] + '/'
print(path_incendios_test)
df_humedad, lst_features_x, df_id = genera_dataframe(ruta_incendio = path_incendios,
                                      ruta_no_incendio = path_no_incendios,
                                      radar= "NDVI",
                                      reshape_x = 23,
                                      reshape_y = 23)

df_humedad_test, lst_features_x, df_id_test = genera_dataframe(ruta_incendio = path_incendios_test,
                                      ruta_no_incendio = path_no_incendios_test,
                                      radar= "NDVI",
                                      reshape_x = 23,
                                      reshape_y = 23)
final = time.time()
print("Ha tardado en generar el dataframe {} minutos".format((final-inicio)/60))

In [ ]:
# Procesos previos
# Separamos los datos
X_train = df_humedad[lst_features_x]
y_train = df_humedad["target"]
X_test = df_humedad_test[lst_features_x]
y_test = df_humedad_test["target"]

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

model = xgb.XGBClassifier(objective='binary:logistic', 
                                      learning_rate = 0.1,
                                      max_depth = 5,
                                      n_estimators=200,
                                      seed=112)

# Entrenamos el modelo
model.fit(X_train, y_train)

# Predecimos los datos de train
y_pred = model.predict(X_test)

# Calculamos los estadísticos
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("El valor de accuracy es: {0}, precision {1}, recall {2} y f1 score {3}".format(
    accuracy, precision, recall, f1
))
df_humedad_test['predicciones_hum'] = y_pred
df_humedad_test['target_hum'] = y_test
df_humedad_test = df_humedad_test[["id","predicciones_hum","target_hum"]]
df_humedad_test = pd.merge(df_humedad_test, df_test, how = 'inner', left_on = 'id', right_on = 'id')
df_humedad_test.head()

In [ ]:
df_humedad_test.to_csv('/content/gdrive/My Drive/incendios_satelite/df_humedad_XGBoost.csv', index=False)

## Poblacion

In [ ]:
df_train_0 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_training_dataclass_0.csv', sep=',')
df_train_1 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_training_dataclass_1.csv', sep=',')
df_train = pd.concat([df_train_0, df_train_1], axis=0)
df_train = df_train.drop_duplicates()

df_test_0 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_validation_dataclass_0.csv', sep=',')
df_test_1 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_validation_dataclass_1.csv', sep=',')
df_test = pd.concat([df_test_0, df_test_1], axis=0)
df_test = df_test.drop_duplicates()

# DATOS DL
# Generamos el dataframe de temperatura
inicio = time.time()
path_no_incendios = path_ruta + lst_variables[2] + '/' + lst_tipo[0] + '/' + lst_clase[0] + '/'
path_incendios = path_ruta + lst_variables[2] + '/' + lst_tipo[0] + '/' + lst_clase[1] + '/'
path_no_incendios_test = path_ruta + lst_variables[2] + '/' + lst_tipo[1] + '/' + lst_clase[0] + '/'
path_incendios_test = path_ruta + lst_variables[2] + '/' + lst_tipo[1] + '/' + lst_clase[1] + '/'
print(path_incendios_test)
df_poblacion, lst_features_x, df_id = genera_dataframe(ruta_incendio = path_incendios,
                                      ruta_no_incendio = path_no_incendios,
                                      radar= "smod",
                                      reshape_x = 23,
                                      reshape_y = 23)

df_poblacion_test, lst_features_x, df_id_test = genera_dataframe(ruta_incendio = path_incendios_test,
                                      ruta_no_incendio = path_no_incendios_test,
                                      radar= "smod",
                                      reshape_x = 23,
                                      reshape_y = 23)
final = time.time()
print("Ha tardado en generar el dataframe {} minutos".format((final-inicio)/60))

In [ ]:
# Procesos previos
# Separamos los datos
X_train = df_poblacion[lst_features_x]
y_train = df_poblacion["target"]
X_test = df_poblacion_test[lst_features_x]
y_test = df_poblacion_test["target"]

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

model = xgb.XGBClassifier(objective='binary:logistic', 
                                      learning_rate = 0.1,
                                      max_depth = 5,
                                      n_estimators=300,
                                      seed=112)

# Entrenamos el modelo
model.fit(X_train, y_train)

# Predecimos los datos de train
y_pred = model.predict(X_test)

# Calculamos los estadísticos
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("El valor de accuracy es: {0}, precision {1}, recall {2} y f1 score {3}".format(
    accuracy, precision, recall, f1
))
df_poblacion_test['predicciones_pob'] = y_pred
df_poblacion_test['target_pob'] = y_test
df_poblacion_test = df_poblacion_test[["id","predicciones_pob","target_pob"]]
df_poblacion_test = pd.merge(df_poblacion_test, df_test, how = 'inner', left_on = 'id', right_on = 'id')
df_poblacion_test.head()

In [ ]:
df_poblacion_test.to_csv('/content/gdrive/My Drive/incendios_satelite/df_poblacion_XGBoost.csv', index=False)

## Real

In [ ]:
df_train_0 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_training_dataclass_0.csv', sep=',')
df_train_1 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_training_dataclass_1.csv', sep=',')
df_train = pd.concat([df_train_0, df_train_1], axis=0)
df_train = df_train.drop_duplicates()

df_test_0 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_validation_dataclass_0.csv', sep=',')
df_test_1 = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_validation_dataclass_1.csv', sep=',')
df_test = pd.concat([df_test_0, df_test_1], axis=0)
df_test = df_test.drop_duplicates()

# DATOS DL
# Generamos el dataframe de temperatura
inicio = time.time()
path_no_incendios = path_ruta + lst_variables[3] + '/' + lst_tipo[0] + '/' + lst_clase[0] + '/'
path_incendios = path_ruta + lst_variables[3] + '/' + lst_tipo[0] + '/' + lst_clase[1] + '/'
path_no_incendios_test = path_ruta + lst_variables[3] + '/' + lst_tipo[1] + '/' + lst_clase[0] + '/'
path_incendios_test = path_ruta + lst_variables[3] + '/' + lst_tipo[1] + '/' + lst_clase[1] + '/'
print(path_incendios_test)

df_real, lst_features_x, df_id = genera_dataframe(ruta_incendio = path_incendios,
                                      ruta_no_incendio = path_no_incendios,
                                      radar= "incendio",
                                      reshape_x = 2100,
                                      reshape_y = 2100)

df_real_test, lst_features_x, df_id_test = genera_dataframe(ruta_incendio = path_incendios_test,
                                      ruta_no_incendio = path_no_incendios_test,
                                      radar= "incendio",
                                      reshape_x = 2100,
                                      reshape_y = 2100)
final = time.time()
print("Ha tardado en generar el dataframe {} minutos".format((final-inicio)/60))

In [ ]:
# Procesos previos
# Separamos los datos
X_train = df_real[lst_features_x]
y_train = df_real["target"]
X_test = df_real_test[lst_features_x]
y_test = df_real_test["target"]

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

model = xgb.XGBClassifier(objective='binary:logistic', 
                                      learning_rate = 0.1,
                                      max_depth = 20,
                                      n_estimators=200,
                                      seed=112)

# Entrenamos el modelo
model.fit(X_train, y_train)

# Predecimos los datos de train
y_pred = model.predict(X_test)

# Calculamos los estadísticos
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("El valor de accuracy es: {0}, precision {1}, recall {2} y f1 score {3}".format(
    accuracy, precision, recall, f1
))
df_real_test['predicciones_real'] = y_pred
df_real_test['target_real'] = y_test
df_real_test = df_real_test[["id","predicciones_real","target_real"]]
df_real_test = pd.merge(df_real_test, df_test, how = 'inner', left_on = 'id', right_on = 'id')
df_real_test.head()

In [ ]:
df_real_test.to_csv('/content/gdrive/My Drive/incendios_satelite/df_poblacion_real.csv', index=False)

## Viento

In [ ]:
df_viento = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_viento.csv', sep=',')
df_viento = df_viento.drop(columns=["target"])
df_viento = df_viento.rename(columns={"col_x": "vel_viento"})
df_viento_test = pd.merge(df_viento, df_test, how = 'inner', left_on = 'id', right_on = 'id')
df_viento_test.head()

## Coordenadas X e Y

In [ ]:
# Procesos previos
# Separamos los datos
var_x = ["X","Y"]
X_train = df_coordenadas_train[var_x]
y_train = df_coordenadas_train["target"]
X_test = df_coordenadas_test[var_x]
y_test = df_coordenadas_test["target"]

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


lr = 0.1
md = 20
ne = 10

model = xgb.XGBClassifier(objective='binary:logistic', 
                                            learning_rate = lr,
                                            max_depth = md,
                                            n_estimators=ne,
                                            seed=112)

# Entrenamos el modelo
model.fit(X_train, y_train)

# Predecimos los datos de train
y_pred = model.predict(X_test)

# Calculamos los estadísticos
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f1)
print("Para los valores XGBoost de learning rate {0}, max_depth {1} y n_estimator {2}, el valor de accuracy es: {3}, precision {4}, recall {5} y f1 score {6}".format(
          lr, md, ne, accuracy, precision, recall, f1
      ))


df_coordenadas_test['predicciones_coord'] = y_pred
df_coordenadas_test['target_coord'] = y_test
df_coordenadas_test = df_coordenadas_test[["id","predicciones_coord","target_coord"]]
df_coordenadas_test.head()

In [ ]:
df_coordenadas_test.to_csv('/content/gdrive/My Drive/incendios_satelite/df_coordenadas_real.csv', index=False)

In [ ]:
# Calculamos los estadísticos
accuracy = accuracy_score(df_coordenadas_test["target_coord"], df_coordenadas_test["predicciones_coord"])
precision = precision_score(df_coordenadas_test["target_coord"], df_coordenadas_test["predicciones_coord"])
recall = recall_score(df_coordenadas_test["target_coord"], df_coordenadas_test["predicciones_coord"])
f1 = f1_score(df_coordenadas_test["target_coord"], df_coordenadas_test["predicciones_coord"])
print("El valor de accuracy es: {0}, precision {1}, recall {2} y f1 score {3}".format(
    accuracy, precision, recall, f1))

## Puertas lógicas

In [ ]:
df_temperatura_test = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_temperatura_XGBoost.csv', sep=',')
df_humedad_test = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_humedad_XGBoost.csv', sep=',')
df_poblacion_test = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_poblacion_XGBoost.csv', sep=',')
df_real_test = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_poblacion_real.csv', sep=',')
df_coordenadas_test = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_coordenadas_real.csv', sep=',')
df_viento_test = pd.read_csv('/content/gdrive/My Drive/incendios_satelite/df_viento.csv', sep=',')

df_temperatura_test = df_temperatura_test.drop_duplicates()
df_humedad_test = df_humedad_test.drop_duplicates()
df_poblacion_test = df_poblacion_test.drop_duplicates()
df_real_test = df_real_test.drop_duplicates()
df_coordenadas_test = df_coordenadas_test.drop_duplicates()
df_viento_test = df_viento_test.drop_duplicates()

items_a_excluir = [16941,18800]
df_temperatura_test = df_temperatura_test[~df_temperatura_test['id'].isin(items_a_excluir)]
df_humedad_test = df_humedad_test[~df_humedad_test['id'].isin(items_a_excluir)]
df_poblacion_test = df_poblacion_test[~df_poblacion_test['id'].isin(items_a_excluir)]
df_real_test = df_real_test[~df_real_test['id'].isin(items_a_excluir)]
df_coordenadas_test = df_coordenadas_test[~df_coordenadas_test['id'].isin(items_a_excluir)]
df_viento_test = df_viento_test[~df_viento_test['id'].isin([16941,18800,17238])]

In [ ]:
print(len(df_temperatura_test))
print(len(df_humedad_test))
print(len(df_poblacion_test))
print(len(df_real_test))
print(len(df_coordenadas_test))
print(len(df_viento_test))

In [ ]:
print(f1_score(df_temperatura_test["target_temp"], df_temperatura_test["predicciones_temp"]))
print(roc_auc_score(df_temperatura_test["target_temp"], df_temperatura_test["predicciones_temp"]))
print(recall_score(df_temperatura_test["target_temp"], df_temperatura_test["predicciones_temp"]))
print(precision_score(df_temperatura_test["target_temp"], df_temperatura_test["predicciones_temp"]))
print(accuracy_score (df_temperatura_test["target_temp"], df_temperatura_test["predicciones_temp"]))
cm = confusion_matrix(df_temperatura_test["target_temp"], df_temperatura_test["predicciones_temp"])
class_names = ['Incendio', 'No incendio',]

conf_mat_df = pd.DataFrame(cm, index=['Verdaderos negativos', 'Falsos positivos'], columns=['Falsos negativos', 'Verdaderos positivos'])

sns.heatmap(conf_mat_df, annot=True, fmt='d', cmap='Blues')

plt.show()

In [ ]:
print(f1_score(df_humedad_test["target_hum"], df_humedad_test["predicciones_hum"]))
print(roc_auc_score (df_humedad_test["target_hum"], df_humedad_test["predicciones_hum"]))

In [ ]:
print(f1_score(df_poblacion_test["target_pob"], df_poblacion_test["predicciones_pob"]))
print(roc_auc_score(df_poblacion_test["target_pob"], df_poblacion_test["predicciones_pob"]))
print(recall_score(df_poblacion_test["target_pob"], df_poblacion_test["predicciones_pob"]))
print(precision_score(df_poblacion_test["target_pob"], df_poblacion_test["predicciones_pob"]))
print(accuracy_score (df_poblacion_test["target_pob"], df_poblacion_test["predicciones_pob"]))
print(classification_report(df_poblacion_test["target_pob"], df_poblacion_test["predicciones_pob"]))

In [ ]:
print(f1_score(df_real_test["target_real"], df_real_test["predicciones_real"]))
print(roc_auc_score (df_real_test["target_real"], df_real_test["predicciones_real"]))

In [ ]:
print(f1_score(df_coordenadas_test["target_coord"], df_coordenadas_test["predicciones_coord"]))
print(roc_auc_score(df_coordenadas_test["target_coord"], df_coordenadas_test["predicciones_coord"]))

In [ ]:
df_conjunto = pd.merge(df_temperatura_test, df_humedad_test, how = 'inner',
                       left_on = 'id', right_on = 'id')
df_conjunto = pd.merge(df_conjunto, df_poblacion_test, how = 'inner',
                       left_on = 'id', right_on = 'id')
df_conjunto = pd.merge(df_conjunto, df_real_test, how = 'inner',
                       left_on = 'id', right_on = 'id')
df_conjunto = pd.merge(df_conjunto, df_viento_test, how = 'inner',
                       left_on = 'id', right_on = 'id')
df_conjunto = pd.merge(df_conjunto, df_coordenadas_test, how = 'inner',
                       left_on = 'id', right_on = 'id')
df_conjunto["target"] = df_conjunto["target_temp"]
df_conjunto["vel_viento"] = (df_conjunto["col_x"] - 149)/76 # Estandarizo
df_conjunto = df_conjunto[["id","predicciones_temp","predicciones_hum",
                           "predicciones_pob","predicciones_real",
                           "predicciones_coord", "vel_viento","target"]]
df_conjunto["suma"] = df_conjunto["predicciones_temp"] + df_conjunto["predicciones_hum"] +\
                      df_conjunto["predicciones_pob"] + df_conjunto["predicciones_real"] + \
                      df_conjunto["predicciones_coord"]

df_conjunto.head()

In [ ]:
print(f1_score(df_conjunto["target"], df_conjunto["predicciones_temp"]))
print(f1_score(df_conjunto["target"], df_conjunto["predicciones_hum"]))
print(f1_score(df_conjunto["target"], df_conjunto["predicciones_pob"]))
print(f1_score(df_conjunto["target"], df_conjunto["predicciones_real"]))
print(f1_score(df_conjunto["target"], df_conjunto["predicciones_coord"]))

In [ ]:
# Calculamos los estadísticos
for i in range(5):
  df_conjunto["predicciones_agrup"] = df_conjunto["suma"].apply(lambda x: 1 if x>=i else 0)
  accuracy = accuracy_score(df_conjunto["target"], df_conjunto["predicciones_agrup"])
  precision = precision_score(df_conjunto["target"], df_conjunto["predicciones_agrup"])
  recall = recall_score(df_conjunto["target"], df_conjunto["predicciones_agrup"])
  f1 = f1_score(df_conjunto["target"], df_conjunto["predicciones_agrup"])
  print("Para {0} variables, el valor de accuracy es: {1}, precision {2}, recall {3} y f1 score {4}".format(
      i+1, accuracy, precision, recall, f1))

## Modelo agrupado

In [ ]:
x_var = ["predicciones_pob"]

# x_var = ["predicciones_pob",        "predicciones_pob","predicciones_coord","vel_viento"]
X = df_conjunto[x_var]
y = df_conjunto["target"]
lst_f1 = []
for i in range(1000):
  # Dividimos los datos en entrenamiento y test
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,)

  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)
  
  model = xgb.XGBClassifier(objective='binary:logistic', 
                                        learning_rate = 0.01,
                                        max_depth = 20,
                                        n_estimators=100,
                                        seed=112)

  # Entrenamos el modelo
  model.fit(X_train, y_train)

  # Predecimos los datos de test
  y_pred = model.predict(X_test)

  # Calculamos los estadísticos
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  
  lst_f1.append(f1)

In [ ]:
from statistics import mean
mean(lst_f1)

In [ ]:
x_var = ["predicciones_temp","predicciones_hum","predicciones_pob",
         "predicciones_pob","predicciones_coord","vel_viento"]
X = df_conjunto[x_var]
y = df_conjunto["target"]

# Dividimos los datos en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Hiperparámetros
learning_rate = [0.001, 0.01, 0.1] # Tasa de aprendizaje
max_depth = [5, 10, 20] # Profundidaz máxima
n_estimators = [10, 50, 100, 200] # Número de árboles

for lr in  learning_rate:
  for md in max_depth:
    for ne in n_estimators:
      model = xgb.XGBClassifier(objective='binary:logistic', 
                                            learning_rate = lr,
                                            max_depth = md,
                                            n_estimators=ne,
                                            seed=112)

      # Entrenamos el modelo
      model.fit(X_train, y_train)

      # Predecimos los datos de train
      y_pred = model.predict(X_test)

      # Calculamos los estadísticos
      accuracy = accuracy_score(y_test, y_pred)
      precision = precision_score(y_test, y_pred)
      recall = recall_score(y_test, y_pred)
      f1 = f1_score(y_test, y_pred)
      print("Para los valores XGBoost de learning rate {0}, max_depth {1} y n_estimator {2}, el valor de accuracy es: {3}, precision {4}, recall {5} y f1 score {6}".format(
          lr, md, ne, accuracy, precision, recall, f1))

In [ ]:
def genera_dataframe(ruta_incendio, ruta_no_incendio, radar, reshape_x,
                     reshape_y):
    
  """
  Esta función parte de las rutas en la que se les haya marcado incendios y no
  incendios y devuelve un dataframe con el ancho solicitado.

  Variables entrada:
    ruta_incendio: Carpeta donde se encuentran las imágenes de incendio
    ruta_no_incendio: Carpeta donde se encuentran las imágenes de no incendio
    radar: tipo de radar (LST_Day, NDVI, et...)
    reshape_x: número de píxeles de ancho que se espera.
    reshape_y: número de píxeles de alto que se espera.

  Ejemplo de llamada:
    df_temperatura = genera_dataframe(ruta_incendio = path_incendios,
                                      ruta_no_incendio = path_no_incendios,
                                      radar= "LST_Day",
                                      reshape_x = 23,
                                      reshape_y = 23)
  """
  lst_archivos_incendios = []
  lst_archivos_no_incendios = []
  print(len(lst_archivos_incendios), len(lst_archivos_no_incendios))
  print(ruta_incendio, ruta_no_incendio)
  # Listado de incendios
  lista_ficheros_incendios = []
  n_varibles_x =reshape_x * reshape_y
  print("Creando lista de incendios a leer...")
  for file in os.listdir(ruta_incendio):
      if radar in file and 'jpeg' in file:
            lst_archivos_incendios.append(file)

  for file in os.listdir(ruta_no_incendio):
      if radar in file and 'jpeg' in file:
          lst_archivos_no_incendios.append(file)  
  print("Leyendo imágenes...")

  contador = 0
    
  for file in  lst_archivos_incendios:
      imagen = cv2.imread(ruta_incendio + file)[0:reshape_x, 0:reshape_y, :]
      image_grey = np.array(cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY))
      if "incendio" in file:
        image_grey = cv2.resize(image_grey, (0,0), fx=0.01, fy=0.01) # Reduzcol 99%
        n_varibles_x =int(reshape_x/100 * reshape_y/100)
      image_grey_reshape = image_grey.reshape(n_varibles_x)
      lst_grey_reshape = image_grey_reshape.tolist()
      lst_grey_reshape.append(2) # Incorporo el target
      lista_ficheros_incendios.append(lst_grey_reshape)
      contador = contador + 1
      print("Imágenes leídas: {}".format(contador))
      #print("Imágenes leídas: {}".format(contador), end='\r')

  for file in  lst_archivos_no_incendios:
      imagen = cv2.imread(ruta_no_incendio + file)[0:reshape_x, 0:reshape_y, :]
      image_grey = np.array(cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY))
      if "incendio" in file:
        image_grey = cv2.resize(image_grey, (0,0), fx=0.01, fy=0.01) # Reduzcol 99%
        n_varibles_x =int(reshape_x/100 * reshape_y/100)
      image_grey_reshape = image_grey.reshape(n_varibles_x)
      lst_grey_reshape = image_grey_reshape.tolist()
      lst_grey_reshape.append(1) # Incorporo el target
      lista_ficheros_incendios.append(lst_grey_reshape)
      contador = contador + 1
      print("Imágenes leídas: {}".format(contador))
    
  # Lo paso a DataFrame
  print("Lo paso a dataframe...")
  lst_features = []
  lst_features_x = []
  for i in range(n_varibles_x):
      lst_features.append("col" + str(i))
      lst_features_x.append("col" + str(i))
  lst_features.append("target")
  print(len(lst_features))
  # Genero el dataframe en sí
  df = pd.DataFrame(data = lista_ficheros_incendios, columns =lst_features )

  lista_id = []
  for item in lst_archivos_incendios:
    id = int(item.split(radar[0])[0])
    lista_id.append(id)
  for item in lst_archivos_no_incendios:
    id = int(item.split(radar[0])[0])
    lista_id.append(id)

  print(len(lst_archivos_incendios), len(lst_archivos_no_incendios), len(lista_id),
        len(df))

  df_id = pd.DataFrame({'id': lista_id})
  df = pd.concat([df, df_id], axis=1)
  print(len(df))
  # Barajo el dataframe
  df=df.sample(frac=1).reset_index(drop=True)

  return df, lst_features_x

# DATOS DL
# Generamos el dataframe de temperatura
inicio = time.time()
df_temperatura, lst_features_x = genera_dataframe(ruta_incendio = path_incendios,
                                      ruta_no_incendio = path_no_incendios,
                                      radar= "LST_Day",
                                      reshape_x = 23,
                                      reshape_y = 23)
final = time.time()
print("Ha tardado en generar el dataframe {} minutos".format((final-inicio)/60))
df_temperatura.head()

In [ ]:
# Generamos procesos previos:
X_train, X_test, y_train, y_test = preprocesado(df_temperatura)

# REGRESIÓN LOGÍSTICA
# Hiperparámetros
C = [0.9, 0.8, 0.7] # Intensidad de la regularización
regularization_methods = ["l1", "l2"] # Métodos de regularización
max_iterations = [100, 1000, 10000, 10000] # Números máximos de iteraciones

temperatura_result_logistica = regresion_logistica(X_train = X_train,
                                              X_test = X_test,
                                              y_train = y_train,
                                              y_test = y_test,
                                              C = C, 
                                regularization_methods = regularization_methods,
                                max_iterations = max_iterations)

# XGBOOST
# Hiperparámetros
learning_rate = [0.001] # Tasa de aprendizaje
max_depth = [10] # Profundidaz máxima
n_estimators = [100] # Número de árboles
temperatura_result_xgboost = func_xgboost(X_train = X_train,
                                      X_test = X_test,
                                      y_train = y_train,
                                      y_test = y_test,
                                      learning_rate = learning_rate,
                                      max_depth = max_depth,
                                    n_estimators = n_estimators)

In [ ]:
temp_best_result_log = max(
    temperatura_result_logistica, key=lambda x: x['f1_score'])
print(temp_best_result_log)
temp_best_result_xgboost = max(
    temperatura_result_xgboost, key=lambda x: x['f1_score'])
print(temp_best_result_xgboost)